# ZI-HDAWG Tutorial 1: Basic pulse generation  

Autho: Nima Leclerc (nleclerc@seas.upenn.edu), PhD Student in Electrical Engineering

Last edited: October 12, 2021

In this tutorial, we will implement standard voltage pulse sequences on the ZI-HDAWG arbitrary waveform generator. This tutorial is based on the documentation found in https://docs.zhinst.com/hdawg_user_manual/tutorial.html. First, ensure the the LabOne software is installed on the PC machine and the AWG is connected to an appropriate router and oscilliscope (or lock-in amplifier. 

To interface with the AWG, we will use the Module $\texttt{zhinst.ziPython}$ for control and data acquisition (DAQ). First, execute the block of code below to establish a connection between the instrument and current PC. 

In [ ]:
import zhinst.ziPython as zi
import zhinst.utils
import numpy as np

dev_id = 'dev8446' ##Device ID for HDAWG 
server_host = 'localhost' ## Host name for server 
server_port = 8004 ## Port number for server 
apilevel_example = 6 ## API number 

##Data aquisition (daq) and device connection  
daq, device, _ = zhinst.utils.create_api_session(dev_id, apilevel_example, server_host=server_host, server_port=server_port)
zhinst.utils.api_server_version_check(daq)
zhinst.utils.disable_everything(daq, device)

The below block of code initializes the experimental settings.

In [ ]:
daq.setInt(f"/{device}/system/awg/channelgrouping", 0) ## initialize channel grouping to 4x2
out_channel = 0 
awg_channel = 0
amplitude = 1.0

exp_setting = [
    ["/%s/sigouts/%d/on" % (device, out_channel), 1],
    ["/%s/sigouts/%d/range" % (device, out_channel), 1],
    ["/%s/awgs/0/outputs/%d/amplitude" % (device, awg_channel), amplitude],
    ["/%s/awgs/0/outputs/0/modulation/mode" % device, 0],
    ["/%s/awgs/0/time" % device, 0],
    ["/%s/awgs/0/userregs/0" % device, 0],
]

In [ ]:
daq.set(exp_setting) ## assign settings to AWG 
daq.sync() ## sync data acquisition module to AWG 
AWG_N = 2000

Provide commands to sequencer. Allocate memory on the instrument, load wave, and play wave. Here, we have a sinusoidal wave  length 2000 

In [ ]:
## Commands to be sent to AWG 
AWG_N = 2000 ## number of points in waveform  
awg_program = textwrap.dedent(
    """\
    const AWG_N = _c1_;
    wave w2 = vect(_w2_);
    playWave(w2);
    """
    )

waveform_2 = np.sin(np.linspace(0, 2 * np.pi, 96)) # Waveform initialized in numpy 
awg_program = awg_program.replace("_w2_", ",".join([str(x) for x in waveform_2]))## Send waveform to AWG
awg_program = awg_program.replace("_c1_", str(AWG_N)) ## Send waveform length to AWG

Now connect to AWG and play program. 

In [ ]:
## Assign changes to AWG and run program via 'set' function 
awgModule = daq.awgModule()
awgModule.set("device", device)
awgModule.execute()

awgModule.set("compiler/sourcestring", awg_program)